In [2]:
# Standard imports
import numpy as np
import pandas as pd
import ast 

# For scraping
import requests
from bs4 import BeautifulSoup

# For adding delays so that we don't spam requests
import time

import string

import re
import json

In [3]:

def get_genre(str_dic):
    genre_list=[]
    for i in str_dic:
        if i == '"genre":':
            j=str_dic.index(i)+1
            while str_dic[j] not in ['],','"actor":','"director":','"creator":','"datePublished":','}</script>]']:
                for punctuation_mark in string.punctuation:
                    str_dic[j]=str_dic[j].replace(punctuation_mark,'')
                if str_dic[j]!="":   
                    genre_list.append(str_dic[j])
                j+=1
            break  

    return genre_list

In [4]:
df_twothousands=pd.read_csv('df_twothousands.csv')
df_twothousands

,tconst,primaryTitle,startYear
0,tt0035423,Kate & Leopold,2001
1,tt0097540,Responso,2004
2,tt0108549,West from North Goes South,2004
3,tt0109173,Auf allen Meeren,2002
4,tt0110476,Master i Margarita,2006
...,...,...,...
27553,tt9600178,Menahi,2008
27554,tt9642628,Zapatistas - Crónica de una Rebelión,2003
27555,tt9660174,Faraway Roots,2002
27556,tt9810488,The Lord of the Rings - The Appendices Part 1:...,2002


In [6]:
scrap_df = pd.DataFrame(columns=['tconst', 'genre', 'plot'])
i=0
j=0
for tconst in df_twothousands['tconst']:
#for tconst in df_movies['tconst'][35494:]:
#for tconst in ['tt0068646','tt0110912','tt0133093','tt0000009','tt0000739','tt0001163','tt0003140','tt0109830']:

    imdb_to_scrap = {'tconst':"", 'genre':[], 'plot':""}
    
    response2=requests.get(f'https://www.imdb.com/title/{tconst}/plotsummary')
    soup2 = BeautifulSoup(response2.content,'html.parser')
        
    if soup2.title.text.strip() != '404 Error - IMDb':
        plot=soup2.find('ul', id="plot-synopsis-content").text.strip()


        if plot!='It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.':

            imdb_to_scrap = {'tconst':tconst, 'genre':[], 'plot':plot}

            # Send a get request and assign the response to a variable
            response1 = requests.get(f'https://www.imdb.com/title/{tconst}/')
            soup1 = BeautifulSoup(response1.content,'html.parser')


            str_dic=str(soup1.findAll('script', {'type':'application/ld+json'})).split()
            imdb_to_scrap['genre']=[get_genre(str_dic)]


            df = pd.DataFrame.from_dict(imdb_to_scrap)
            scrap_df = scrap_df.append(df, ignore_index=True)
            

            #follow progress: print tconst every 500 iteration
            i+=1
            if i%25==0:
                print(tconst)

            


scrap_df

tt0139654
tt0165982
tt0181875
tt0198021
tt0211070
tt0219699
tt0234215
tt0242029
tt0246578
tt0252444
tt0259446
tt0265208
tt0270919
tt0277607
tt0283530
tt0289408
tt0294357
tt0298578
tt0304141
tt0310793
tt0316396
tt0319343
tt0327084
tt0331811
tt0337978
tt0342097
tt0347304
tt0351817
tt0360139
tt0363589
tt0366627
tt0369441
tt0373926
tt0377091
tt0380741
tt0384680
tt0387952
tt0393882
tt0398237
tt0402399
tt0406038
tt0410377
tt0414852
tt0417072
tt0419449
tt0421229
tt0424227
tt0427470
tt0431197
tt0435434
tt0438315
tt0442933
tt0446046
tt0450259
tt0453671
tt0456165
tt0460740
tt0462371
tt0465502
tt0468704
tt0472131
tt0475331
tt0478090
tt0479893
tt0482606
tt0486572
tt0489235
tt0492956
tt0496413
tt0499556
tt0764260
tt0780076
tt0790623
tt0796307
tt0804522
tt0809533
tt0817225
tt0825232
tt0832320
tt0841150
tt0852726
tt0862467
tt0872036
tt0885415
tt0893325
tt0903665
tt0912583
tt0925260
tt0940723
tt0952682
tt0963208
tt0974515
tt0983991
tt0990413
tt0997147
tt1003538
tt1019452
tt1029095
tt1034331
tt1043451


,tconst,genre,plot
0,tt0035423,"[Comedy, Fantasy, Romance]","In 1876, Leopold Alexis Elijah Walker Gareth T..."
1,tt0118694,"[Drama, Romance]",Two couples Mr and Mrs Chan ( Mrs Chan played ...
2,tt0118852,"[Drama, contentRating, R]",Harry Levine (Al Pacino) is a washed-up writer...
3,tt0118926,"[Crime, Drama, Thriller]",Detective Augustin Rejas (Javier Bardern) atte...
4,tt0120630,"[Animation, Adventure, Comedy, Drama, Family]",The Tweedys are a troubled middle-aged couple ...
...,...,...,...
3492,tt4627732,[Drama],The plot of the film is around the Nagorniy Ka...
3493,tt5230508,[Comedy],An enormously creative and witty comedic film ...
3494,tt5466580,[Documentary],Tokom visemjesenih napora Armije BiH za zauzim...
3495,tt7098636,[Documentary],The clitoris is the only part of the body whic...


In [8]:
scrap_df

,tconst,genre,plot
0,tt0035423,"[Comedy, Fantasy, Romance]","In 1876, Leopold Alexis Elijah Walker Gareth T..."
1,tt0118694,"[Drama, Romance]",Two couples Mr and Mrs Chan ( Mrs Chan played ...
2,tt0118852,"[Drama, contentRating, R]",Harry Levine (Al Pacino) is a washed-up writer...
3,tt0118926,"[Crime, Drama, Thriller]",Detective Augustin Rejas (Javier Bardern) atte...
4,tt0120630,"[Animation, Adventure, Comedy, Drama, Family]",The Tweedys are a troubled middle-aged couple ...
...,...,...,...
3492,tt4627732,[Drama],The plot of the film is around the Nagorniy Ka...
3493,tt5230508,[Comedy],An enormously creative and witty comedic film ...
3494,tt5466580,[Documentary],Tokom visemjesenih napora Armije BiH za zauzim...
3495,tt7098636,[Documentary],The clitoris is the only part of the body whic...


In [9]:
scrap_df.to_csv (r'scrap_df_twothousandstens.csv', index = False, header=True)